In [7]:
import torch
import torchvision
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

In [8]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # flatten all dims except batch
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
def load_data():
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    batch_size = 4
    trainset = torchvision.datasets.CIFAR10(
        root='./data', 
        train=True,
        download=True, 
        transform=transform
    )
    trainloader = torch.utils.data.DataLoader(
        trainset, 
        batch_size=batch_size,
        shuffle=True, 
        num_workers=2
    )
    testset = torchvision.datasets.CIFAR10(
        root='./data', 
        train=False,
        download=True, 
        transform=transform
    )
    testloader = torch.utils.data.DataLoader(
        testset, 
        batch_size=batch_size,
        shuffle=False, 
        num_workers=2
    )
    classes = ('plane', 'car', 'bird', 'cat','deer', 
               'dog', 'frog', 'horse', 'ship', 'truck')
    return trainloader, testloader, classes

## Training code has code to save the models trained locally
- Researcher manually manages the artifacts and performs versioning
- Need to write code to get a holistic view of all runs
- Stores results locally, making collaboration harder
- Manual effort required to run this training on a VM

In [10]:
def train_model(trainloader, lr, save_path='model.pth', epochs=1):
    net = Net()
    
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else: 
        device = torch.device('cpu')
    
    net.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
    for epoch in range(epochs):
        for i, data in enumerate(trainloader, 0):
            inputs = data[0].to(device)
            labels = data[1].to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Save the entire model
    torch.save(net, save_path)
    
    return net

In [11]:
def run_inference_and_tests(net, testloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct // total

In [12]:
data = load_data()
trainloader = data[0]
testloader = data[1]
classes = data[2]

Files already downloaded and verified
Files already downloaded and verified


## Let's say for the purpose of the POC, we only have one parameter to train

In [13]:
learning_rate = [0.01, 0.001]
best_model = None
best_score = -1
for lr in learning_rate:
    model = train_model(trainloader, lr, 'model_' + str(lr*1000) + '.pth')
    accuracy = run_inference_and_tests(model, testloader)
    if accuracy > best_score:
        best_score = accuracy
        best_model = model

In [ ]:
print(f"Best model accuracy was {best_score}%.")